In [123]:
import re
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup


In [124]:
url  = 'https://books.toscrape.com/catalogue/category/books/classics_6/index.html'
url1 = 'https://books.toscrape.com/catalogue/category/books/science-fiction_16/index.html'
url2 = 'https://books.toscrape.com/catalogue/category/books/humor_30/index.html'
url3 = 'https://books.toscrape.com/catalogue/category/books/business_35/index.html'

headers = {'User-Agente': 'Mozilla/5.0 (Macintosh; Intel Max OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [125]:
classicurl = requests.get(url, headers = headers )
classicurl.text
science_fictionurl = requests.get(url1, headers = headers )
science_fictionurl.text
humorurl = requests.get(url2, headers = headers )
humorurl.text
businessurl = requests.get(url3, headers = headers )
businessurl.text;

In [270]:
headers = {'User-Agente': 'Mozilla/5.0 (Macintosh; Intel Max OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

def soup (url):
    url = requests.get(url, headers = headers)
    url.text
    soup = BeautifulSoup(url.text, 'html.parser')
    books = soup.find('ol', class_ = 'row')
    return books

In [283]:
# Name
def name (books):
    books_list = books.find_all('a')
    books_name = [p.get('title') for p in books_list]
    books_name = list(filter(None,books_name))
    books_name = pd.DataFrame(books_name, columns = ['name'])
    return books_name

In [282]:
# Price
def price (books):
    books_price = books.find_all('p', class_ = 'price_color')
    books_price = [p.get_text() for p in books_price]
    books_price = pd.DataFrame(books_price,columns=['price'])
    return books_price

In [250]:
# Rating
def rating (books):
    books_list = books.find_all('p',class_ = 'star-rating')
    books_rating = [p.get('class') for p in books_list]
    books_rating = pd.DataFrame(books_rating, columns= ['','rating'])
    books_rating = books_rating.drop(columns = '')
    books_rating['rating'] = books_rating['rating'].apply(lambda x: '1' if x == 'One' else '2' if x == 'Two' else '3' if x == 'Three' else '4' if x == 'Four' else 'Five')
    return books_rating

In [259]:
# URL
def url_stock (books):
    books_url = books.find_all('a')
    books_url = [p.get('href') for p in books_url]
    books_url = list(set(books_url))
    books_url = [p.replace('../../../','/') for p in books_url]
    books_url = pd.DataFrame(books_url, columns = ['url'])
    
    return books_url

In [301]:
# Stock
def stock (books_url):
    stock_number_list = []
    stock_name_list = []
    for i in range(len(books_url)):
        url = 'https://books.toscrape.com/catalogue/' + books_url.loc[i, 'url']
        headers = {'User-Agente': 'Mozilla/5.0 (Macintosh; Intel Max OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        stock_number = requests.get(url, headers = headers )
        stock_number.text
        soup = BeautifulSoup(stock_number.text, 'html.parser')
        stock_number = soup.find_all('p', class_ = 'instock availability')
        stock_number = [p.get_text() for p in stock_number][0]
        stock_number_list.append(stock_number)
        stock_number_list = [p.replace('\n\n    \n        ','') for p in stock_number_list]
        stock_number_list = [p.replace('\n    \n','') for p in stock_number_list]
    
        stock_name = soup.find_all('h1')
        stock_name = [p.get_text() for p in stock_name]
        stock_name_list.append(stock_name)
        
        stock_name = pd.DataFrame(stock_name_list)
        stock_name.columns = ['name']    
        stock_number = pd.DataFrame(stock_number_list)
        stock_number.columns = ['stock']
    
        return stock_name, stock_number
    

In [298]:
df_partial1 = pd.concat([books_name,books_price,books_rating],axis=1)
df_partial2 = pd.concat([stock_name,stock_number],axis = 1)
df_final = pd.merge(df_partial1,df_partial2,how = 'left' , on = 'name')

NameError: name 'books_rating' is not defined